## Importing Explanatory Variables

N.B. We use the 'nordpool' module, for scraping day-ahead price data; and 'requests', to to query weather data via the Frost API in a HTTP format.

---

Move up one level to the project root to access entire repository. Specify the year to investigate.

In [4]:
#We need to access modules from the project root, so we select this directory.
import os, sys

# Move 1 level up from notebooks/ to project root
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

year = 2024

This cell queries the Frost API for observation data. All stations in the county of Agder (first found by querying 'sources') are queried for their max air temperature, mean wind speed, and total duration of sunshine per day. Measurements are aggregated day by day and we treat these values as representative of the weather conditions in Southern Norway.

In [6]:
from data_import_functions.weather import Frost_weather_data
Frost_weather_data(year)

Data retrieved from frost.met.no!


elementId,max(air_temperature P1D),mean(wind_speed P1D),sum(duration_of_sunshine P1D)
referenceTime,,,
2024-01-01,-1.729268,8.550,0.00
2024-01-02,-3.585366,9.520,0.00
2024-01-03,-7.492683,10.000,0.00
2024-01-04,-9.385714,6.165,0.00
2024-01-05,-12.385714,4.450,0.00
...,...,...,...
2024-12-27,5.423810,1.680,0.00
2024-12-28,5.350000,3.320,0.05
2024-12-29,7.602381,6.615,2.55


Now we use the NordPool API to extract the daily day-ahead price data for NO2 and various other regions (which the NO2 price is coupled to via interconnectors) over the same time period.

In [8]:
from data_import_functions.eu_spots import day_ahead_NP_prices
day_ahead_NP_prices(['NO2','GER','DK1','NL'], year)

region,DK1,GER,NL,NO2
date,,,,
2024-01-01,32.94,16.18,18.95,46.63
2024-01-02,53.59,53.07,63.06,61.42
2024-01-03,45.97,45.22,51.43,66.27
2024-01-04,85.41,85.41,84.68,89.44
2024-01-05,91.37,91.37,91.22,90.95
...,...,...,...,...
2024-12-27,121.71,121.71,121.43,35.33
2024-12-28,122.30,122.64,122.31,30.92
2024-12-29,50.28,105.28,111.95,27.46


Also adding in the UK day-ahead price (which is not traded on Nordpool) but is interconnected to NO2 via the North Sea Link (NSL). Historical European power market price data downloaded from Ember.

In [10]:
from data_import_functions.uk_spot import UK_prices
UK_prices(year)

Date
2024-01-01     56.15
2024-01-02     64.83
2024-01-03     81.08
2024-01-04     93.16
2024-01-05     96.25
               ...  
2024-12-27    128.33
2024-12-28    122.70
2024-12-29    107.89
2024-12-30    106.83
2024-12-31     63.24
Name: UK, Length: 366, dtype: float64

Gas power plants also feature in Norway's generation stack with stations such as Karsto (420 MW capacity) and Mongstad (280 MW capacity). Although usually, hydropower is enough to fulfil Norway's grid demands, occasionally, gas power plants also contribute. So, we expect (perhaps when reservoir levels are low) that spot gas prices affect day-ahead power prices, as this (spark) spread influences the decision-making of gas generators.

As Dutch TTF (Title Transfer Facility) Nautral Gas is the most liquid European market, we use it as a benchmark for gas prices in Europe. We are unable to access 2024 historical spot TTF data, so instead, we use front-month futures data (price of futures with expiry in the following month) as a proxy (imported from YahooFinance with units EUR/MWh).

We investigate whether front-month is a good proxy for spot in: TTF_gas_price_future_spot_comparison.ipynb. We find a strong correlation. However, front-month is much less volatile than spot (probably because market news does not have as immediate of an impact). We conclude that front-month is a good enough representation.

In [12]:
from data_import_functions.ttf_gas_price import ttf_gas
ttf_gas(year)

2024-01-02    30.572001
2024-01-03    32.794998
2024-01-04    33.410000
2024-01-05    34.550999
2024-01-08    31.580000
                ...    
2024-12-24    45.730999
2024-12-26    45.730999
2024-12-27    47.730999
2024-12-30    47.632999
2024-12-31    48.889000
Name: Gas Price, Length: 251, dtype: float64

As HEP is the main source of power generation in Norway, we add in data regarding the monthly water storage levels in reservoirs in Southern Norway. The measurement of energy stored is provided in MWh and we convert to TWh: this represents the energy equivalent of water stored.

We suspect that the amount stored, and in particular the anomaly (diffrence between current volume stored and the historical average), impacts the willingness of HEP stations to let flow and therefore their bid, which often sets the clearing price.

In [14]:
from data_import_functions.reservoir_levels import reservoir_levels
reservoir_levels(year).head(10)

,Reservoir stores,Storage anomaly
Week,,
1,22.374192,-0.443833
2,21.148020,-1.141372
3,19.996998,-1.570836
4,19.653578,-1.141271
5,19.339328,-0.680329
6,18.420454,-0.767393
7,17.421780,-1.069974
8,16.517006,-1.505716
9,15.653768,-1.519732
